In [ ]:
from keras.models import load_model
import scipy.io as sio 
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Input, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.python.client import device_lib

# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
gpu_devices = tf.config.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
    
np.random.seed(1)
list_test=[]
list_target=[]
def read_input(fileName):
    i=sio.loadmat(fileName)
    TTrain=i['TTrain']
    XTrain=i['XTrain']
    TTest=i['TTest']

    XTrain=XTrain.reshape(-1)
    TTrain=TTrain.reshape(-1)
    #list_target.append(TTrain)
    test=np.transpose(XTrain).reshape(-1)
    test2=np.transpose(TTrain).reshape(-1)
    TTest=np.transpose(TTest)
    
    list_target.append(test2)

    
    for i in range(len(test)):
        test[i]=np.transpose(np.array(test[i]))
        list_test.append(test[i])
        
#There are total files (1-8 except 6)
trainFileNum=8

#File test number
#textFileNumber=8
i=1    
while i <= trainFileNum:
    #Skip file number 6
    if(i==6):
        i+=1
        continue
    else:
        fileName="inputFeatures"+str(i)+".mat"
        read_input(fileName)
        i+=1
        
list_test=np.vstack(list_test)
scaler = StandardScaler()
scaler=scaler.fit(list_test)
scaled_list_test=scaler.transform(list_test)
#fileName='inputFeatures1.mat'

#TTest=np.transpose(TTest)
#TTrain=np.transpose(TTrain)
#XTrain=np.transpose(XTrain)
#XTest=np.transpose(XTest)

# for j in range(len(test2)):  
#     test2[j]=np.transpose(np.array(test2[j]))
#     list_test.append(test2[j])  

print(scaled_list_test.shape)

#print(scaled_list_test)
#print(scaled_list_test.shape)

XT_s_stack=list()
i=0
while i+50 <= (len(scaled_list_test)):
    XT_s_stack.append(scaled_list_test[i:i+50])
    i=i+50

#print(listTest)
XT_s_stack=np.stack(XT_s_stack)

list_target=np.hstack(list_target)
print(list_target.shape)


#cutPoint=np.shape(TTest.reshape(-1))[0]
#print(np.shape(TTest.reshape(-1))[0])

#XTrain_s=XT_s_stack[:-cutPoint,:,:]
#XTest_s=XT_s_stack[-cutPoint:,:,:]
print(XT_s_stack.shape)

XTrain_s=XT_s_stack
TTrain=list_target

length = 50
n_features = 4
model = Sequential()
model.add(LSTM(150,input_shape=(length,n_features)))
#model.add(tf.keras.layers.RNN(tf.keras.layers.LSTMCell(150)),input_shape=(length,n_features))
#model.add(Dense(16, activation= 'elu', kernel_initializer='normal'))

#model.add(Dense(1, activation='linear'))
model.add(Dense(1))

#opt=Adam(learning_rate=0.02)
#model.compile(optimizer=opt, loss='mean_squared_logarithmic_error')

model.compile(optimizer='adam', loss='mean_squared_logarithmic_error')
model.summary()

#with tf.device('/gpu:0'):
model.fit(XTrain_s,np.array(TTrain),epochs=1000,verbose=2)
    

#TTest=TTest.reshape(len(TTest),1)
#Test=np.transpose(TTest)
# #print(np.array(TTest))

# test=np.transpose(XTrain).flatten()
# #print(test)
# list_test=list()
# for i in test:
#     #test[i]=np.transpose(np.array(i))
#     list_test.append(np.array(i))

# list_test=np.vstack(list_test)
# x=np.stack(list_test)



(609550, 4)
(12191,)
(12191, 50, 4)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 150)               93000     
                                                                 
 dense (Dense)               (None, 1)                 151       
                                                                 
Total params: 93,151
Trainable params: 93,151
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
381/381 - 8s - loss: 0.9541 - 8s/epoch - 22ms/step
Epoch 2/1000
381/381 - 3s - loss: 0.1986 - 3s/epoch - 7ms/step
Epoch 3/1000
381/381 - 3s - loss: 0.1383 - 3s/epoch - 7ms/step
Epoch 4/1000
381/381 - 3s - loss: 0.1334 - 3s/epoch - 9ms/step
Epoch 5/1000
381/381 - 3s - loss: 0.1252 - 3s/epoch - 7ms/step
Epoch 6/1000
381/381 - 3s - loss: 0.1209 - 3s/epoch - 8ms/step
Epoch 7/1000
381/381 - 3s - loss:

In [ ]:
print(XTrain_s.shape)
print(np.array(TTrain).shape)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#list_target_noob=[]


plt.figure(figsize=(20,30))

def test_input(fileName,idx):
    i=sio.loadmat(fileName)
    TTest=i['TTest']
    XTest=i['XTest']

    XTest=np.array(XTest.reshape(-1))
    TTest=np.transpose(TTest)
   
    testStack=list()
    for i in XTest:
        testStack.append(np.transpose(i))
    testStack=np.stack(testStack)    

    with tf.device('/cpu:0'):
        Y_hat=model.predict(testStack,verbose=0)
    print(np.shape(Y_hat))
    print(np.shape(TTest))
    plt.subplot(4,2,idx)
    plt.plot(TTest,color='green',label='Actual')
    plt.plot(Y_hat,color='blue',label='Predicted')
    plt.xlabel('Number')
    plt.ylabel('VO2')        
    plt.title(f'Subj_{idx}: Actual vs Predicted')
    
    #test2=np.transpose(TTrain).reshape(-1)
    #list_target_noob.append(test2)

i=1
idx=1
testFileNum=8
while i <= trainFileNum:
    #Skip file number 6
    if(i==6):
        i+=1
        idx+=1
        continue
    else:
        fileName="inputFeatures"+str(i)+".mat"
        test_input(fileName,idx)       
        idx+=1
        i+=1
        
    
plt.legend()
plt.savefig('5000pochs_Multiple_Subj_Training.jpg',bbox_inches='tight',dpi=150)
plt.show()
#print(np.hstack(list_target_noob).shape)

In [ ]:
# i=sio.loadmat("inputFeatures1.mat")
# TTest=i['TTest']
# XTest=i['XTest']
# print(np.array(TTest).shape)
# Y_hat=model.predict(XTest)
# print(Y_hat.shape)

In [ ]:
# print(Y_hat.shape)

In [ ]:
# test_fileName="inputFeatures8_m.mat"
# i=sio.loadmat(test_fileName)
# TTest=i['TTrain']
# XTest=i['XTrain']
# XTest_s=np.array(XTest.reshape(-1))
# print(XTest_s)

# testStack=list()
# for i in XTest_s:
#     testStack.append(np.transpose(i))
# testStack=np.stack(testStack)    

# with tf.device('/cpu:0'):
#     Y_hat=model.predict(testStack)

In [ ]:
# print(Y_hat.shape)
# TTest=np.transpose(TTest)
# print(TTest.shape)

In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib.pyplot import figure

# plt.plot(TTest,color='green',label='Actual')
# plt.plot(Y_hat,color='blue',label='Predicted')
# plt.xlabel('Number')
# plt.ylabel('VO2')
# plt.title('Actual vs Predicted')
# plt.legend()
# #plt.figure(figsize=(100,100))

# plt.show()
# fig = plt.gcf()
# fig.set_size_inches(100.5, 30.5)
# #fig.savefig('multiple_subject_input_pred_7.png', dpi=150)

In [ ]:
model.save('5000pochs_Multiple_Subj_Training.h5')